In [44]:
import os
import yaml
from zipfile import ZipFile
import pooch

with open("./proj_cfg.yml", "r", encoding="utf-8") as proj_cfg:
    PROJ_PATH = yaml.load(proj_cfg, Loader=yaml.FullLoader)["proj_path"]
os.chdir(PROJ_PATH)


# Atlas path
ATLAS_PATH = os.path.join(PROJ_PATH, "assets/atlas/")
ATLAS_WRITE_PATH = os.path.join(ATLAS_PATH, "P14-MEAN")
CIRCUIT_PATH = os.path.join(ATLAS_WRITE_PATH, "hierarchy.json")

In [41]:
# connect to  Atlas on Harvard dataverse
data = pooch.create(
    base_url="doi:10.7910/DVN/QREN2T",
    path=ATLAS_PATH,
)
data.load_registry_from_doi()

# unit-test that the zip. file exists
data.registry_files == ["atlas.zip"]

# download
datafile = data.fetch("atlas.zip")

# unzip
with ZipFile(ATLAS_PATH + "atlas.zip", "r") as f:
    f.extractall(ATLAS_PATH)

# unit-test
os.path.isdir(os.path.join(ATLAS_PATH, "P14-MEAN"))

False

In [64]:
import voxcell
from voxcell.nexus.voxelbrain import Atlas
import numpy as np

voxels = voxcell.VoxelData.load_nrrd(ATLAS_WRITE_PATH + "/brain_regions.nrrd")
print(voxels.voxel_dimensions)  # prints array([100., 100., 100.], dtype=float32)

atlas = Atlas.open(ATLAS_WRITE_PATH)

brain_regions = atlas.load_data("brain_regions")
rm = atlas.load_region_map()

# count the number of voxels in the VIS region, and all its descendents
ids = rm.find("VIS", "acronym", with_descendants=True)
np.count_nonzero(np.isin(brain_regions.raw, list(ids)))

[ 38.7348  38.7348 -38.7348]


set()

# references

https://github.com/BlueBrain/snap/blob/master/doc/source/notebooks/01_circuits.ipynb